In [7]:
from diffusers import DDPMScheduler
import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp
import torch.distributed as dist
from torch.utils.data import Dataset, DataLoader
import torchaudio
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os

from VQ_model import get_model
from diffusion_model import VQ_diffwave
import params as params
from IPython.display import Audio

In [3]:
vq_model = get_model(data_variance = None, inference = True)
vq_model.load_state_dict(torch.load('VQ_pth/model.pth'))

<All keys matched successfully>

In [4]:
with torch.no_grad():
    wav, sr = torchaudio.load('../dataset/ESC-50-master/audio/1-103298-A-9.wav')
    wav = torchaudio.transforms.Resample(sr, 16000)(wav)
    wav = wav.unsqueeze(1)
    z = vq_model(wav)
    output = vq_model.decoder(z)
    sample = output.squeeze(0)
    print(sample.shape)
    sample_wav = torchaudio.save('sample.wav', sample.cpu(), 16000)

torch.Size([1, 80000])


In [22]:
wav, sr = torchaudio.load('sample_test.wav')
spec = torchaudio.transforms.Spectrogram()(wav)


tensor([[[1.8222e+03, 1.0867e+01, 7.5818e+02,  ..., 1.3297e+01,
          5.5233e-02, 6.7544e+01],
         [1.0959e+03, 1.0555e+02, 4.2179e+02,  ..., 2.0232e+01,
          3.2413e+01, 5.9449e+01],
         [2.8181e+02, 8.9268e+00, 2.1007e+01,  ..., 1.0755e+01,
          2.6052e+01, 1.3188e+01],
         ...,
         [1.1023e-04, 1.3692e-08, 1.5618e-08,  ..., 1.1548e-08,
          3.1744e-08, 1.6095e-07],
         [1.1238e-04, 8.0217e-10, 2.6339e-09,  ..., 1.1182e-08,
          1.3706e-08, 9.6164e-08],
         [1.1409e-04, 4.1746e-09, 5.6762e-09,  ..., 6.7462e-09,
          5.7199e-12, 2.0327e-08]]])